#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbgraos_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date   producao  disponibilidade   tipo
0  2017  114732101         17427206   soja
1  2018  117912450         16126368   soja
2  2019  114316829         17515920   soja
3  2020  121820949         18258981   soja
4  2021  134799179         18984300   soja
5  2022  120701031         21037669   soja
6  2017   97910658         33959879  milho
7  2018   82366531         34777936  milho
8  2019  101126409         35895207  milho
9  2020  103990935         37191638  milho


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             12 non-null     int64 
 1   producao         12 non-null     int64 
 2   disponibilidade  12 non-null     int64 
 3   tipo             12 non-null     object
dtypes: int64(3), object(1)
memory usage: 512.0+ bytes


### Avaliando a disponibilidade de soja

In [4]:
df.drop(['producao'], axis = 1,inplace=True)
df = df[df['tipo']=='soja']
df.head(10)


,date,disponibilidade,tipo
0,2017,17427206,soja
1,2018,16126368,soja
2,2019,17515920,soja
3,2020,18258981,soja
4,2021,18984300,soja
5,2022,21037669,soja


In [5]:
df.drop(['tipo'], axis = 1,inplace=True)


In [6]:
df.set_index('date',inplace=True)

In [7]:
df=df.sort_index(axis=0)
print(df.head(10))

      disponibilidade
date                 
2017         17427206
2018         16126368
2019         17515920
2020         18258981
2021         18984300
2022         21037669


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [8]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,disponibilidade
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(6, 1)"
5,Transformed data shape,"(6, 1)"
6,Transformed train set shape,"(5, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [9]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [10]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [11]:
top4 = exp.compare_models(n_select = 4)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
croston,Croston,0.7466,0.7328,910025.6460,910025.6460,0.0489,0.0506,0.0933
naive,Naive Forecaster,0.7514,0.7450,952644.0000,952644.0000,0.0527,0.0544,2.6100
exp_smooth,Exponential Smoothing,0.8595,0.8591,1128472.2737,1128472.2737,0.0636,0.0672,1.6633
ets,ETS,0.9224,0.9209,1203988.3370,1203988.3370,0.0677,0.0716,0.0533
grand_means,Grand Means Forecaster,0.9768,0.9623,1209043.5278,1209043.5278,0.0656,0.0680,4.1600
theta,Theta Forecaster,1.0477,1.0424,1344041.1882,1344041.1882,0.0753,0.0800,0.0400
arima,ARIMA,1.1343,1.1182,1402497.4486,1402497.4486,0.0766,0.0798,0.4067
polytrend,Polynomial Trend Forecaster,1.1720,1.1659,1506151.4444,1506151.4444,0.0841,0.0893,3.4467
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.2933,1.2853,1656197.7003,1656197.7003,0.0922,0.0978,0.2767
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.3868,1.3806,1795311.5985,1795311.5985,0.0999,0.1066,0.1033


Processing:   0%|          | 0/112 [00:00<?, ?it/s]

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [12]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

## ARIMA model

The ARIMA model is a model based on the Autoregression moving average (ARMA), 
but since the ARMA model is only a model for stationary time series, it is implemented as an ARIMA model including normalizing the abnormal data.

In [13]:
huber_cds_dt = exp.create_model('huber_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = huber_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.0682,2.0682,2690390.0000,2690390.0000,0.1536,0.1664
1,2019.0000,1.5003,1.4994,2018135.2213,2018135.2213,0.1105,0.1170
2,2020.0000,0.5919,0.5742,677409.5743,677409.5743,0.0357,0.0363
Mean,nan,1.3868,1.3806,1795311.5985,1795311.5985,0.0999,0.1066
SD,nan,0.6080,0.6157,836763.7356,836763.7356,0.0487,0.0536


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.



## lightgbm_cds_dt Light Gradient Boosting w/ Cond. Deseasonalize & Detrending 
The series is tested for seasonality using the test outlined in A&N. 
If deemed seasonal, the series is seasonally adjusted using a classical multiplicative decomposition before applying the theta method. The resulting forecasts are then reseasonalised.

In [14]:
lightgbm_cds_dt = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.0682,2.0682,2690390.0000,2690390.0000,0.1536,0.1664
1,2019.0000,1.0194,1.0189,1371301.5052,1371301.5052,0.0751,0.0780
2,2020.0000,0.7924,0.7687,906901.5957,906901.5957,0.0478,0.0489
Mean,nan,1.2933,1.2853,1656197.7003,1656197.7003,0.0922,0.0978
SD,nan,0.5557,0.5630,755461.0528,755461.0528,0.0449,0.0499


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## POLYTREND

In [15]:

polytrend  = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.0682,2.0682,2690390.0000,2690390.0000,0.1536,0.1664
1,2019.0000,0.8527,0.8523,1147102.3333,1147102.3333,0.0628,0.0649
2,2020.0000,0.5950,0.5772,680962.0000,680962.0000,0.0359,0.0365
Mean,nan,1.1720,1.1659,1506151.4444,1506151.4444,0.0841,0.0893
SD,nan,0.6424,0.6478,858734.5030,858734.5030,0.0504,0.0557


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## theta   Theta Forecaster 

In [16]:

theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 9)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:25:56
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Theta Forecaster


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.0673,2.0673,2689180.5937,2689180.5937,0.1535,0.1663
1,2019.0000,0.5571,0.5568,749421.2221,749421.2221,0.0410,0.0419
2,2020.0000,0.5186,0.5031,593521.7490,593521.7490,0.0313,0.0318
Mean,nan,1.0477,1.0424,1344041.1882,1344041.1882,0.0753,0.0800
SD,nan,0.7211,0.7250,953284.2096,953284.2096,0.0555,0.0612


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,1.947788e+07
2023,1.974022e+07
2024,2.000256e+07
2025,2.026490e+07
2026,2.052724e+07
2027,2.078958e+07
2028,2.105192e+07
2029,2.131426e+07
2030,2.157660e+07


In [17]:

ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,1.8449,1.8449,2399924.1500,2399924.1500,0.1370,0.1471
1,2019.0000,0.7788,0.7784,1047684.3827,1047684.3827,0.0574,0.0591
2,2020.0000,0.1436,0.1393,164356.4783,164356.4783,0.0087,0.0086
Mean,nan,0.9224,0.9209,1203988.3370,1203988.3370,0.0677,0.0716
SD,nan,0.7019,0.7036,919334.5050,919334.5050,0.0529,0.0572


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,1.947788e+07
2023,1.974022e+07
2024,2.000256e+07
2025,2.026490e+07
2026,2.052724e+07
2027,2.078958e+07
2028,2.105192e+07
2029,2.131426e+07
2030,2.157660e+07


## polytrend

In [18]:
polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})



,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,2.0682,2.0682,2690390.0000,2690390.0000,0.1536,0.1664
1,2019.0000,0.8527,0.8523,1147102.3333,1147102.3333,0.0628,0.0649
2,2020.0000,0.5950,0.5772,680962.0000,680962.0000,0.0359,0.0365
Mean,nan,1.1720,1.1659,1506151.4444,1506151.4444,0.0841,0.0893
SD,nan,0.6424,0.6478,858734.5030,858734.5030,0.0504,0.0557


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ETS
The ETS algorithm is especially useful for datasets with seasonality and other prior assumptions about the data. 

ETS computes a weighted average over all observations in the input time series dataset as its prediction. 

The weights are exponentially decreasing over time, rather than the constant weights in simple moving average methods. 

The weights are dependent on a constant parameter, which is known as the smoothing parameter.

In [19]:
ets_dt = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,1.8449,1.8449,2399924.1500,2399924.1500,0.1370,0.1471
1,2019.0000,0.7788,0.7784,1047684.3827,1047684.3827,0.0574,0.0591
2,2020.0000,0.1436,0.1393,164356.4783,164356.4783,0.0087,0.0086
Mean,nan,0.9224,0.9209,1203988.3370,1203988.3370,0.0677,0.0716
SD,nan,0.7019,0.7036,919334.5050,919334.5050,0.0529,0.0572


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,1.947788e+07
2023,1.974022e+07
2024,2.000256e+07
2025,2.026490e+07
2026,2.052724e+07
2027,2.078958e+07
2028,2.105192e+07
2029,2.131426e+07
2030,2.157660e+07


## ALL GRAPHS

In [20]:
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets_dt, plot = 'forecast',
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
               

## Avaliando a disponibilidade do milho 

In [21]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbgraos_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date   producao  disponibilidade   tipo
0  2017  114732101         17427206   soja
1  2018  117912450         16126368   soja
2  2019  114316829         17515920   soja
3  2020  121820949         18258981   soja
4  2021  134799179         18984300   soja
5  2022  120701031         21037669   soja
6  2017   97910658         33959879  milho
7  2018   82366531         34777936  milho
8  2019  101126409         35895207  milho
9  2020  103990935         37191638  milho


In [22]:
df.drop(['producao'], axis = 1,inplace=True)
df = df[df['tipo']=='milho']
df.head(10)

,date,disponibilidade,tipo
6,2017,33959879,milho
7,2018,34777936,milho
8,2019,35895207,milho
9,2020,37191638,milho
10,2021,39126269,milho
11,2022,40894968,milho


In [23]:
df.drop(['tipo'], axis = 1,inplace=True)

In [24]:
df.set_index('date',inplace=True)

In [25]:
df=df.sort_index(axis=0)
print(df.head(10))

      disponibilidade
date                 
2017         33959879
2018         34777936
2019         35895207
2020         37191638
2021         39126269
2022         40894968


In [26]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,disponibilidade
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(6, 1)"
5,Transformed data shape,"(6, 1)"
6,Transformed train set shape,"(5, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [27]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [28]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [29]:
top4 = exp.compare_models(n_select = 4)


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

In [30]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})


### ets ETS 

In [31]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(ets, fh = 9)


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2022,3.886811e+07
2023,3.968617e+07
2024,4.050422e+07
2025,4.132228e+07
2026,4.214034e+07
2027,4.295839e+07
2028,4.377645e+07
2029,4.459451e+07
2030,4.541256e+07


### exp_smooth  Exponential Smoothing  

In [32]:
exp_smooth = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ETS

In [33]:
naive = exp.create_model('naive')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = naive, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,1.3658,1.3658,1117271.0000,1117271.0000,0.0311,0.0316
1,2019.0000,1.3398,1.3240,1296431.0000,1296431.0000,0.0349,0.0355
2,2020.0000,1.7959,1.7665,1934631.0000,1934631.0000,0.0494,0.0507
Mean,nan,1.5005,1.4854,1449444.3333,1449444.3333,0.0385,0.0393
SD,nan,0.2092,0.1995,350788.7803,350788.7803,0.0079,0.0082


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.



## exp_smooth


In [34]:
croston = exp.create_model('croston')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = croston, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### ALL GRAPHS

In [35]:
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [36]:
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 9)


,y_pred
2022,1.947788e+07
2023,1.974022e+07
2024,2.000256e+07
2025,2.026490e+07
2026,2.052724e+07
2027,2.078958e+07
2028,2.105192e+07
2029,2.131426e+07
2030,2.157660e+07
